In [1]:
import requests
import os
import csv
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from random import randint
from datetime import datetime

In [6]:
### 3901.csv in Dataframe (EGID)

egid = pd.read_csv('EGID/3901/3901.csv', delimiter=';')

# Spalte URL hinzufügen
egid['URL'] = 'https://www.housing-stat.ch/regbl/resources/public/geb_public/' + egid['EGID'].astype('str') + '?lang=de'


In [2]:
# URL herunterladen, als pdf abspeichern

# alle URLS in Liste, für download in Schlaufe
url_liste = egid.values.tolist()

urls = []
url_egid = {}

for url_list in url_liste:
    # Dictionary je Zeile aus Dataframe
    einzelne_egid = url_list[0]
    einzelne_url = url_list[13]
    
    url_egid = {'EGID': einzelne_egid, 'URL': einzelne_url}
    urls.append(url_egid)

# jede URL runterladen und als pdf abspeichern
for url in tqdm(urls):
    
    inhalt = requests.get(url['URL'], allow_redirects=True)
    time.sleep(5)
    with open('PDF/GEB_PUBLIC' + str(url['EGID']) + '.pdf', 'wb') as pdffile:
        pdffile.write(inhalt.content)
        
        

NameError: name 'egid' is not defined

In [13]:
# fehlerhafte URL herunterladen, als pdf abspeichern, MANUELL!!

# alle URLS in Liste, für download in Schlaufe


    
    
# alle URLS in Liste, für download in Schlaufe
url_liste = egid.values.tolist()

urls = []
url_egid = {}

for url_list in url_liste:
    # Dictionary je Zeile aus Dataframe
    einzelne_egid = url_list[0]
    einzelne_url = url_list[13]
    
    url_egid = {'EGID': einzelne_egid, 'URL': einzelne_url}
    urls.append(url_egid)

# jede URL runterladen und als pdf abspeichern
for url in tqdm(urls):
    
    if ('GEB_PUBLIC' + str(url['EGID']) + '.pdf') not in os.listdir('PDF/'): #pdf nicht schon da
    
    
        inhalt = requests.get(url['URL'], allow_redirects=True)
        time.sleep(5)
        with open('PDF/GEB_PUBLIC' + str(url['EGID']) + '.pdf', 'wb') as pdffile:
            pdffile.write(inhalt.content)



 85%|████████▍ | 4329/5107 [05:38<00:04, 181.40it/s]


100%|██████████| 5107/5107 [05:43<00:00, 14.87it/s] 


In [2]:
# PDF auslesen
# EGID, Gebäudenummer, Koordinaten etc.

import os
import textract
from tqdm import tqdm
import pandas as pd


# leere Liste für alle Gebäude
alle_gebaeude = []
fehlerhafte_pdf = []

# PDF in Liste
pdfs = [f for f in os.listdir('PDF/') if f.endswith('.pdf') or f.endswith('.PDF')]

for pdf in tqdm(pdfs):

    try:
        text = textract.process('PDF/' + pdf, method='pdfminer', encoding='ISO-8859-1')
        text = text.decode("ISO-8859-1").replace("\n", "xxxxxxxxxx").split("xxxxxxxxxx")



        #minidict grad mit den gewünschten elementen

        startposition = text.index('ParzNr.')
        GrundstueckNummer = text[2 + startposition] 
        
        startposition = text.index('Eidg. Gebäudeidentifikator')
        EGID = text[2 + startposition]
        
        startposition = text.index('X-Wert')
        GebaeudeKoordinateX = text[-6 + startposition]
        
        startposition = text.index('Y-Wert')
        GebaeudeKoordinateY = text[2 + startposition]
        
        startposition = text.index('Baujahr')
        Baujahr = text[1 + startposition]
           
        startposition = text.index('Gebäudekategorie')
        Gebäudekategorie = text[9 + startposition] + " " + text[10 + startposition]
           
        startposition = text.index('Gebäudeklasse')
        Gebäudeklassenummer = text[2 + startposition]
           
        startposition = text.index('Gebäudeklasse')
        Gebäudeklasse = text[9 + startposition] + " " + text[10 + startposition]

        startposition = text.index('Gebäudefläche (in m²)')
        Gebäudefläche = text[7 + startposition]

        startposition = text.index('Anz. Geschosse')
        AnzGeschosse = text[2 + startposition]

        startposition = text.index('Strasse')
        GebaeudeStrasse = text[1 + startposition]

        startposition = text.index('Strasse')
        GebaeudeHausnummer = text[4 + startposition]
             
        startposition = text.index('PLZ')
        GebaeudePLZ = text[1 + startposition]
             
        startposition = text.index('Ort')
        GebaeudeOrt = text[1 + startposition]
             
            
            
        gebaeude = {'Grundstueck-Nummer': GrundstueckNummer,
                   'EGID': EGID,
                   'Gebaeude-Koordinate X': GebaeudeKoordinateX,
                   'Gebaeude-Koordinate Y': GebaeudeKoordinateY,
                   'Gebaeude-Baujahr': Baujahr,
                   'Gebaeude-Kategorie': Gebäudekategorie,
                   'Gebaeude-Klasse-Nummer': Gebäudeklassenummer,
                   'Gebaeude-Klasse': Gebäudeklasse,
                   'Gebaeude-Flaeche': Gebäudefläche,
                   'Gebaeude-Geschosse': AnzGeschosse,
                   'Gebaeude-Strasse': GebaeudeStrasse,
                   'Gebaeude-Hausnummer': GebaeudeHausnummer,
                   'Gebaeude-PLZ': GebaeudePLZ,
                   'Gebaeude-Ort': GebaeudeOrt}

        alle_gebaeude.append(gebaeude)
        
    except:
        fehlerhafte_pdf.append(pdf)
        
 


# csv abspeichern
file = pd.DataFrame(alle_gebaeude)            
headernames = ['Grundstueck-Nummer', 'EGID', 'Gebaeude-Koordinate X', 'Gebaeude-Koordinate Y',
               'Gebaeude-Baujahr', 'Gebaeude-Kategorie', 'Gebaeude-Klasse-Nummer',
               'Gebaeude-Klasse', 'Gebaeude-Flaeche', 'Gebaeude-Geschosse', 'Gebaeude-Strasse',
               'Gebaeude-Hausnummer', 'Gebaeude-PLZ', 'Gebaeude-Ort']

file.to_csv('Gebäude_Chur.csv', header=headernames, sep=";", index=False)



with open("fehlerhafte_pdf.csv", 'w+') as csvfile:
    
    writer = csv.writer(csvfile, delimiter='\n')
    writer.writerows([fehlerhafte_pdf])


100%|██████████| 5011/5011 [32:46<00:00,  2.55it/s]
